In [1]:
import subprocess
import os
import argparse
import pandas as pd
import requests
from sqlalchemy import create_engine
from dotenv import load_dotenv
import logging
from sqlalchemy import text
import os
import subprocess
import duckdb
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import datetime

load_dotenv()
DATABASE_URL = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
engine = create_engine(DATABASE_URL)

In [2]:
engine 
start_year=2024 
end_year=2024 
parquet_folder = 'data/raw/nflfastr/'

In [275]:
weekly_df = pd.read_sql("SELECT * FROM weekly_stats", engine)
players_df = pd.read_sql("SELECT * FROM players", engine)
teams_df = pd.read_sql('SELECT * FROM teams', engine)
games_df = pd.read_sql('SELECT * FROM games', engine)
depth_df = pd.read_sql('SELECT * FROM depth_chart', engine)
injuries_df = pd.read_sql('SELECT * FROM injuries', engine)
schedule_df = pd.read_sql("SELECT season, week, stadium, game_date FROM games", engine)
base_features_df = pd.read_sql("SELECT * FROM player_weekly_features", engine)

In [3]:
weekly_stats_df = pd.read_sql('SELECT * FROM weekly_stats', engine)
games_df = pd.read_sql('SELECT * FROM games', engine)
weather_df = pd.read_sql('SELECT * FROM weather', engine)
weekly_stats_features = weekly_stats_df[['player_id',
                'player_name',
                'player_display_name',
                'position',
                'team_abbreviation',
                'season',
                'opponent_team',
                'week',
                'attempts',
                'completions',
                'passing_yards',
                'passing_tds',
                'targets',
                'receptions',
                'carries',
                'rushing_yards',
                'rushing_tds',
                'receiving_yards',
                'receiving_tds',
                'target_share',
                'fantasy_points']].sort_values(by = ['player_name','season','week']).drop_duplicates().reset_index(drop = True)

# Adding weather data to base feature table
player_games = weekly_stats_features.merge(
    games_df,
    how = 'left',
    on = ['season', 'week']
)

# Filter rows where team_abbreviation matches either home or away team
player_games = player_games[
    (player_games['team_abbreviation'] == player_games['home_team']) |
    (player_games['team_abbreviation'] == player_games['away_team'])
]

player_games_with_weather = player_games.merge(
    weather_df,
    how = 'left',
    on = ['season', 'week', 'stadium']
)

player_games_with_weather['home_game'] = (
    player_games_with_weather['team_abbreviation'] == player_games_with_weather['home_team']
)

final_df = player_games_with_weather[
    weekly_stats_features.columns.tolist() + ['stadium', 'temperature', 'precipitation', 'wind_speed', 'dome', 'home_game']
].drop_duplicates().reset_index(drop = True)

In [4]:
final_df

,player_id,player_name,player_display_name,position,team_abbreviation,season,opponent_team,week,attempts,completions,...,receiving_yards,receiving_tds,target_share,fantasy_points,stadium,temperature,precipitation,wind_speed,dome,home_game
0,32104,A.Abdullah,Ameer Abdullah,RB,MIN,2021,CIN,1,0,0,...,15.0,0,0.042553,1.90,Paycor Stadium,75.4,0.000,18.1,False,False
1,32104,A.Abdullah,Ameer Abdullah,RB,MIN,2021,ARI,2,0,0,...,0.0,0,0.031250,0.00,State Farm Stadium,72.0,0.000,0.0,True,False
2,32104,A.Abdullah,Ameer Abdullah,RB,MIN,2021,SEA,3,0,0,...,0.0,0,NaN,2.40,U.S. Bank Stadium,72.0,0.000,0.0,True,True
3,32104,A.Abdullah,Ameer Abdullah,RB,MIN,2021,CLE,4,0,0,...,2.0,0,0.027778,0.20,U.S. Bank Stadium,72.0,0.000,0.0,True,True
4,32104,A.Abdullah,Ameer Abdullah,RB,MIN,2021,DET,5,0,0,...,0.0,0,NaN,0.20,U.S. Bank Stadium,72.0,0.000,0.0,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20876,37013,Z.Wilson,Zach Wilson,QB,NYJ,2023,LV,10,39,23,...,0.0,0,NaN,13.92,Allegiant Stadium,72.0,0.000,0.0,True,False
20877,37013,Z.Wilson,Zach Wilson,QB,NYJ,2023,BUF,11,15,7,...,0.0,0,NaN,6.74,Highmark Stadium,40.2,0.000,18.2,False,False
20878,37013,Z.Wilson,Zach Wilson,QB,NYJ,2023,HOU,14,36,27,...,0.0,0,NaN,19.24,MetLife Stadium,52.8,0.708,13.4,False,True
20879,37013,Z.Wilson,Zach Wilson,QB,NYJ,2023,HOU,14,36,27,...,0.0,0,NaN,19.24,MetLife Stadium,38.3,0.000,11.1,False,True
